# Image Feature Extraction (Feature Database Generation)

In [1]:
import os
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pickle
from utils import *
from Features import *

### Get the image database

In [2]:
DatabasePath = "database"
Categories = [os.path.split(x)[-1] for x in glob.glob(os.path.join(DatabasePath, "*"))]
ImagePaths = []
for C in Categories:
    ImagePaths.append(glob.glob(os.path.join(DatabasePath, C, "*.jp*g")))

### Initialize Image Feature Database

In [3]:
# Initialize with previous saved feature data base
# print("Loading feature database ...")
# with open("FeatureDatabase.pkl","rb") as f:
#     FeatureDatabase = pickle.load(f)
# print()

In [4]:
# Initialize with empty feature database
print("Initializing feature database ...")
FeatureDatabase = {}
for C in Categories:
    FeatureDatabase[C] = []

print("Total Categories: %d" % len(Categories))
for C in range(len(Categories)):
    CurCategory = Categories[C]
    for I in ImagePaths[C]:
        RawImage = LoadImage(I)
        Gray = np.expand_dims(cv2.cvtColor(RawImage, cv2.COLOR_RGB2GRAY), axis=-1)
        HSV = cv2.cvtColor(RawImage, cv2.COLOR_RGB2HSV)
        YUV = cv2.cvtColor(RawImage, cv2.COLOR_RGB2YUV)
        Lab = cv2.cvtColor(RawImage, cv2.COLOR_RGB2Lab)
        CurImageDict = {}
        CurImageDict['Path'] = I
        CurImageDict['RGB Image'] = RawImage
        CurImageDict['Gray Image'] = Gray
        CurImageDict['HSV Image'] = HSV
        CurImageDict['YUV Image'] = YUV
        CurImageDict['Lab Image'] = Lab
        CurImageDict['Gabor Images'] = GaborFilteredImages(Gray)
        FeatureDatabase[CurCategory].append(CurImageDict)
    print("█", end='')
print()
print()

Initializing feature database ...
Total Categories: 30
██████████████████████████████

### Add Image Features

In [5]:
print("Adding image features ...")

CatIter = 1
TotalCatIter = len(Categories)
for C in range(TotalCatIter):
    CurCategory = Categories[C]
    print("Category %d/%d: %s" % (CatIter, TotalCatIter, CurCategory))
    CatIter += 1
    for I in FeatureDatabase[CurCategory]:
        # Color Histograms
        I["Global RGB Histogram"] = GlobalColorHistogram(I['RGB Image'], n_bins=[4, 4, 4], ranges=[0, 256, 0, 256, 0, 256])
        I["Local RGB Histogram"] = LocalColorHistogram(I['RGB Image'], n_bins=[4, 4, 4], ranges=[0, 256, 0, 256, 0, 256])
        I['Global Gray Histogram'] = GlobalColorHistogram(I['Gray Image'], channels=[0], n_bins=[32], ranges=[0, 256])
        I['Local Gray Histogram'] = LocalColorHistogram(I['Gray Image'], channels=[0], n_bins=[32], ranges=[0, 256])
        I['Global HSV Histogram'] = GlobalColorHistogram(I['HSV Image'], n_bins=[36, 2, 2], ranges=[0, 180, 0, 256, 0, 256])
        I['Local HSV Histogram'] = LocalColorHistogram(I['HSV Image'], n_bins=[36, 2, 2], ranges=[0, 180, 0, 256, 0, 256])
        I['Global YUV Histogram'] = GlobalColorHistogram(I['YUV Image'], n_bins=[32, 2, 2], ranges=[0, 256, 0, 256, 0, 256])
        I['Local YUV Histogram'] = LocalColorHistogram(I['YUV Image'], n_bins=[32, 2, 2], ranges=[0, 256, 0, 256, 0, 256])  
        I['Global Lab Histogram'] = GlobalColorHistogram(I['Lab Image'], n_bins=[32, 2, 2], ranges=[0, 256, 0, 256, 0, 256])
        I['Local Lab Histogram'] = LocalColorHistogram(I['Lab Image'], n_bins=[32, 2, 2], ranges=[0, 256, 0, 256, 0, 256])  
        
        # Color Moments
        I["Global RGB Moment"] = GlobalColorMoment(I['RGB Image'])
        I["Local RGB Moment"] = LocalColorMoment(I['RGB Image'])
        I['Global Gray Moment'] = GlobalColorMoment(I['Gray Image'], channels=[0])
        I['Local Gray Moment'] = LocalColorMoment(I['Gray Image'], channels=[0])
        I['Global HSV Moment'] = GlobalColorMoment(I['HSV Image'])
        I['Local HSV Moment'] = LocalColorMoment(I['HSV Image'])
        I['Global YUV Moment'] = GlobalColorMoment(I['YUV Image'])
        I['Local YUV Moment'] = LocalColorMoment(I['YUV Image'])  
        I['Global Lab Moment'] = GlobalColorMoment(I['Lab Image'])
        I['Local Lab Moment'] = LocalColorMoment(I['Lab Image'])  
        
        # Color Auto-Correlogram
        I["RGB Auto-Correlogram"] = AutoCorrelogram(I['RGB Image'], n_bins=[4, 4, 4], ranges=[0, 256, 0, 256, 0, 256])
        I["Gray Auto-Correlogram"] = AutoCorrelogram(I['Gray Image'], channels=[0], n_bins=[32], ranges=[0, 256])
        I["HSV Auto-Correlogram"] = AutoCorrelogram(I['HSV Image'], n_bins=[16, 2, 2], ranges=[0, 180, 0, 256, 0, 256])
        I["YUV Auto-Correlogram"] = AutoCorrelogram(I['YUV Image'], n_bins=[16, 2, 2], ranges=[0, 256, 0, 256, 0, 256])
        I["Lab Auto-Correlogram"] = AutoCorrelogram(I['Lab Image'], n_bins=[16, 2, 2], ranges=[0, 256, 0, 256, 0, 256])
    
        # Gabor Feature
        I['Gabor Features'] = ExtractGaborFeature(I['Gabor Images'])
        I['Gabor Global Histogram'] = GaborGlobalHistogram(I['Gabor Images'])
        I['Gabor Local Histogram'] = GaborLocalHistogram(I['Gabor Images'])
        
        # Local Binary Pattern
        I['Local Binary Pattern'] = LocalBinaryPattern(I['Gray Image'])
        I['Grid Local Binary Pattern'] = GridLocalBinaryPattern(I['Gray Image'])
        
        # Histogram of Oriented Gradients
        I['HOG'] = HistogramofOrientedGradients(I['Gray Image'])
        I['PHOG'] = PyramidHOG(I['Gray Image'])
        
        # Shape Index
        I['Shape Index'] = ShapeIndex(I['Gray Image'])
        
        # SIFT
        I['SIFT'] = GetSIFT(I['Gray Image'], limit=None)
        I['SIFT Limited'] = GetSIFT(I['Gray Image'], limit=512)
        I['Dense SIFT'] = GetDenseSIFT(I['Gray Image'])
        I['Pyramid SIFT'] = GetPyramidSIFT(I['Gray Image'])
        print("█", end='')
        
    print()

Adding image features ...
Category 1/30: aloe_vera_gel
████████████████████
Category 2/30: baby_shoes
████████████████████
Category 3/30: bicycle
███████████████████
Category 4/30: bottle
████████████████████
Category 5/30: bracelet
████████████████████
Category 6/30: cartoon_purse
████████████████████
Category 7/30: chair
████████████████████
Category 8/30: children_dress
████████████████████
Category 9/30: clock
████████████████████
Category 10/30: cup
████████████████████
Category 11/30: drum
████████████████████
Category 12/30: garment
████████████████████
Category 13/30: gge_snack
████████████████████
Category 14/30: glasses
████████████████████
Category 15/30: goggles
████████████████████
Category 16/30: hand_cream
████████████████████
Category 17/30: ice_cream
████████████████████
Category 18/30: korean_snack
████████████████████
Category 19/30: leather_purse
████████████████████
Category 20/30: men_clothes
████████████████████
Category 21/30: minnie_dress
████████████████████
C

### Dump Feature Database

In [6]:
with open("FeatureDatabase.pkl","wb") as f:
    pickle.dump(FeatureDatabase, f)